### Simulation Code for Rate Networks

In [1]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
import numba as nb
import pandas as pd
import statsmodels.api as sm
from joblib import Parallel, delayed
import os
import time as tm

#### Definition of Functions

In [ ]:
#name of the folder to store the data
def folder_name(N, N_steps, N_cut, dt):
    return  'data_N{}_N_steps{}_N_cut{}_dt{}/'.format(N, N_steps, N_cut, dt)

# Activation function of the Rate Network
@nb.jit
def phi(x):
    return np.tanh(x)

#Runge Kutta IV integration step
@nb.jit
def RK4_step(x_t, dt, J):

    k1 = rhs(x_t, J)
    k2 = rhs(x_t + dt/2 * k1, J)
    k3 = rhs(x_t + dt/2 * k2, J)
    k4 = rhs(x_t + dt * k3, J)
    
    return x_t + dt/6 * (k1 + 2*k2 + 2*k3 + k4)

#Rate Network model ODE
@nb.jit
def rhs(x_t, J):
    Nn = len(x_t)
    f = np.zeros(Nn)
    for i in range(Nn):
        tmp = 0
        for j in range(Nn):
            tmp = tmp + J[i, j]* phi(x_t[j])
        
        f[i] = -x_t[i] + tmp

    return f

#Runge Kutta IV integrator
@nb.jit
def RK4(x0, dt, N_steps, J):
    x = np.zeros((len(x0), N_steps))
    x[:, 0] = x0
    for n in range(1, N_steps):
        x[:, n] = RK4_step(x[:, n-1], dt, J)
    return x

#Half-Width at Half-Maximum Estimator
def HWHM(C):
    half = 0.5
    idx = np.argmin(np.abs(C - half))

    return idx


#Process of simulation
def rate_network_sim(params, save_sim = True):

    #Unpack parameters
    [N, g, p_dist, mu, sigma, mean, N_steps, N_cut, dt, n, index] = params

    #Generate random connectivity matrix
    J = np.random.normal(0, g/np.sqrt(N), (N, N))    #Rate network model

    if p_dist == 'homogeneous':
        S = np.ones(N)*mean

    elif p_dist == 'heterogeneous':
        S = np.random.lognormal(mu, sigma, N)

    #Final interaction matrix    
    J = J - np.diag(np.diag(J)) +np.diag(S)


    #Generate initial conditions
    x0 = np.random.uniform(-10, 10, N)

    #Run the simulation
    sim = RK4(x0, dt, N_steps, J)

    
    #Compute the autocorrelation time
    tau = []
    for j in range(N):
        # Compute the autocorrelation function for a range of lags
        C = sm.tsa.acf(phi(sim[j, N_cut:]), nlags=N_steps-N_cut, fft=False)
        #C = autocorrelation(sim[j, :], discard = N_cut, max = N_steps-N_cut)
        tau.append(HWHM(C)*dt)
    
    sigma_tau = np.std(tau)
    

    folder = folder_name(N, N_steps, N_cut, dt)
    

    if save_sim:
        np.savez(folder + 'simulations/' + 'sim_s_dist_{}_mu{}_sigma{}_mean{}_g{}_n{}_index{}.npz'.format(p_dist, mu, sigma, mean, g, n, index), sim = sim, J = J, S = S, tau = np.array(tau))

    return index

#### Simulations parameters

In [3]:
#number of assemblies
N = 400 

#gain of the network
g_list = [2.5]

#Number of repetitions for each parameter set
N_rep = 8

#distribution of assembly strengths
p_dist_list = ['homogeneous']#['homogeneous', 'heterogeneous']

#mu of lognormal
mu_list = [None, 0.6]

#sigma of lognormal
sigma_list = [None, 0.6]

#mean of P(s)
mean_list = [2.2, None]

#P_s_parameters
P_s_params = list(zip(p_dist_list, mu_list, sigma_list, mean_list))

#integration parametes 
N_steps = 61_000
N_cut = 1000
dt = 0.05
timesteps = np.arange(N_steps+1)
time = dt*timesteps

#### List of parameters and folders to save the data

In [5]:
for p_dist, mu, sigma, mean in P_s_params:
    print('Distribution: ', p_dist, 'mu: ', mu, 'sigma: ', sigma, 'mean: ', mean)

Distribution:  homogeneous mu:  None sigma:  None mean:  2.2


In [6]:
# computing the list of parameters
params = []
counter = 0
for g in g_list:
    for n in range(N_rep):
        for p_dist, mu, sigma, mean in P_s_params:
            params.append([N, g, p_dist, mu, sigma, mean, N_steps, N_cut, dt, n, counter])
            counter = counter +1 

#folders to store the data
folder = folder_name(N, N_steps, N_cut, dt)
os.mkdir(folder)
os.mkdir(folder + 'simulations')
os.mkdir(folder + 'plots')

#saving the list of parameters
params_to_save = np.array(params)
np.save(folder+'parameters.npy', params_to_save)

In [6]:
#folders to store the data
folder = folder_name(N, N_steps, N_cut, dt)

#loading the parameters list
params = np.load(folder+'parameters.npy', allow_pickle=True).tolist()

#setting the index of the parameter list to simulate
index_inf = 249
index_max = len(params)


#### Code to run the simulations in parallel

In [7]:
#simuations to run
params = params[index_inf:index_max]

#list to store the runned simulations
sim_saved = []

print('Number of tasks:', len(params))
data = Parallel(n_jobs = -1, verbose = 11)(delayed(rate_network_sim)(par) for par in params)

#storing the index of the runned parameters
data = np.array(data)
np.save(folder+'indexes_run_{}.npy'.format(tm.time()), data)

Number of tasks: 7


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 127.3min
[Parallel(n_jobs=-1)]: Done   2 out of   7 | elapsed: 127.7min remaining: 319.2min
[Parallel(n_jobs=-1)]: Done   3 out of   7 | elapsed: 128.1min remaining: 170.8min
[Parallel(n_jobs=-1)]: Done   4 out of   7 | elapsed: 129.6min remaining: 97.2min
[Parallel(n_jobs=-1)]: Done   5 out of   7 | elapsed: 130.6min remaining: 52.2min
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 131.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   7 out of   7 | elapsed: 131.8min finished
